# **6.1**

In [45]:
import numpy as np
import random

In [46]:

def remainder_of_polynomial_division(dividend, divisor):
    """
    Вычисляет остаток от деления многочлена (делимого) на многочлен (делитель)
    с использованием алгоритма побитового исключающего ИЛИ (XOR).

    Параметры:

    dividend :
        Массив, представляющий коэффициенты делимого многочлена,
        где индекс соответствует степени (от старшей к младшей).

    divisor :
        Массив, представляющий коэффициенты делителя многочлена,
        где индекс соответствует степени (от старшей к младшей).
        Должен быть ненулевым и иметь степень меньше или равную степени делимого.

    Возвращает:

    np.array
        Массив, представляющий остаток от деления,
        где индекс соответствует степени (от старшей к младшей).
        Если остаток равен нулю, возвращается пустой массив.
    """
    # Создаем копию делимого для проведения модификаций
    current_remainder = list(dividend)

    # Получаем длину делителя
    divisor_length = len(divisor)

    # Продолжаем деление, пока степень остатка не станет меньше степени делителя
    while len(current_remainder) >= divisor_length:
        # Рассчитываем сдвиг для выравнивания делителя с текущим остатком
        shift_position = len(current_remainder) - divisor_length

        # Применяем побитовое исключающее ИЛИ (XOR) между делителем, сдвинутым на shift_position позиций, и остатком
        for i in range(divisor_length):
            current_remainder[shift_position + i] ^= divisor[i]

        # Удаляем нули в конце остатка для уменьшения его степени
        while current_remainder and current_remainder[-1] == 0:
            current_remainder.pop()

    return np.array(current_remainder)

In [47]:
def multiply_polynomials(A, B):
    """
    Умножает два многочлена, представленных в виде списков коэффициентов,
    используя побитовое исключающее ИЛИ (XOR) для сложения коэффициентов.

    Аргументы:
    A : np.array
        Список коэффициентов первого многочлена, где индекс соответствует степени
        переменной.

    B : np.array
        Список коэффициентов второго многочлена, где индекс соответствует степени
        переменной.

    Возвращает:
    np.array
        Список коэффициентов произведения двух многочленов. Длина результата равна
        сумме степеней множителей минус один.
    """
    length_a = len(A)  # Длина первого многочлена
    length_b = len(B)  # Длина второго многочлена

    # Инициализация массива для хранения результата произведения
    product_result = np.zeros(length_a + length_b - 1, dtype=int)

    # Проходим по каждому коэффициенту второго многочлена
    for index in range(length_b):
        if B[index] != 0:  # Проверяем, не равен ли коэффициент нулю
            # Умножаем текущий коэффициент B на весь многочлен A
            # Сдвигаем результат на 'index' позиций вправо
            product_result[index:index + length_a] ^= A.astype(int)

    return product_result

In [48]:

def generate_and_correct_error(a, g, error_rate):
    """
    Генерирует сообщение с ошибками, исправляет их и проверяет, удалось ли восстановить исходное сообщение.

    Параметры:
    a (list): Список коэффициентов исходного сообщения, представленного в виде многочлена.

    g (list): Список коэффициентов порождающего полинома, который используется для кодирования.

    error_rate (int): Частота ошибок, которую нужно сгенерировать.
                      1 - одна ошибка, 2 - две ошибки, любое другое значение - указанное количество ошибок.

    Возвращает:
    None: Функция выводит на экран информацию о процессе генерации и исправления ошибок,
          но не возвращает никаких значений.
    """
    print("Исходное сообщение:      ", a)
    print("Порождающий полином:     ", g)

    # Умножаем исходное сообщение на порождающий полином для получения отправленного сообщения
    v = multiply_polynomials(a, g)
    print("Отправленное сообщение:  ", v)

    # Копируем отправленное сообщение для внесения ошибок
    w = v.copy()
    error = np.zeros(len(w), dtype=int)  # Инициализируем массив для ошибок

    # Генерация ошибок в зависимости от заданной частоты
    if error_rate == 1:
        index = random.randint(0, len(w) - 1)  # Генерируем случайный индекс для одной ошибки
        error[index] = 1
    elif error_rate == 2:
        index1 = random.randint(0, len(w) - 2)  # Генерируем первый индекс для двух ошибок
        index2 = index1 + random.choice([1, 2])  # Генерируем второй индекс, чтобы он не совпадал с первым
        error[index1] = 1
        error[index2] = 1
    else:
        # Генерируем случайные индексы для указанного количества ошибок
        error_indices = random.sample(range(len(w)), error_rate)
        for index in error_indices:
            error[index] = 1

    # Вносим ошибки в сообщение
    w = (w + error) % 2
    print("Сообщение с ошибкой:     ", w)

    # Вычисляем остаток от деления с использованием порождающего полинома
    s = remainder_of_polynomial_division(w, g)

    # Определяем шаблоны ошибок в зависимости от частоты ошибок
    error_patterns = [[1]] if error_rate == 1 else [[1, 1, 1], [1, 0, 1], [1, 1], [1]]

    idx = 0
    found = False

    # Поиск соответствия остатков с шаблонами ошибок
    while not found:
        for pattern in error_patterns:
            if np.array_equal(s, pattern):  # Проверяем, совпадает ли остаток с шаблоном
                found = True
                break
        if not found:
            # Если совпадений не найдено, сдвигаем остаток и увеличиваем индекс
            s = remainder_of_polynomial_division(multiply_polynomials(s, np.array([0, 1])), g)
            idx += 1

    # Создаем временный массив для исправления ошибок
    temp = np.zeros(len(w), dtype=int)
    if idx == 0:
        temp[0] = 1  # Устанавливаем первый элемент, если idx == 0
    else:
        temp[len(temp) - idx - 1] = 1  # Устанавливаем элемент с учетом индекса

    # Умножаем остаток на временный массив для получения исправления
    e = multiply_polynomials(s, temp)[:len(w)]
    message = (w + e) % 2  # Исправляем сообщение
    print("Исправленное сообщение:  ", message)

    # Проверяем, удалось ли восстановить исходное сообщение
    if np.array_equal(v, message):
        print("Ошибка исправлена успешно")
    else:
        print("Ошибка исправлена неудачно")

In [49]:
# Введем входное сообщение a и порождающий полином g = 1 + x^2 + x^3
a = np.array([1, 0, 0 ,1])
g = np.array([1, 0, 1, 1])

In [50]:
generate_and_correct_error(a, g, 1) # исследование для однократной ошибки

Исходное сообщение:       [1 0 0 1]
Порождающий полином:      [1 0 1 1]
Отправленное сообщение:   [1 0 1 0 0 1 1]
Сообщение с ошибкой:      [1 0 0 0 0 1 1]
Исправленное сообщение:   [1 1 0 0 0 1 1]
Ошибка исправлена неудачно


In [51]:
generate_and_correct_error(a, g, 2) # исследование для двухкратной ошибки

Исходное сообщение:       [1 0 0 1]
Порождающий полином:      [1 0 1 1]
Отправленное сообщение:   [1 0 1 0 0 1 1]
Сообщение с ошибкой:      [1 1 0 0 0 1 1]
Исправленное сообщение:   [1 1 0 0 0 0 1]
Ошибка исправлена неудачно


In [52]:
generate_and_correct_error(a, g, 3) # исследование для трехкратной ошибки

Исходное сообщение:       [1 0 0 1]
Порождающий полином:      [1 0 1 1]
Отправленное сообщение:   [1 0 1 0 0 1 1]
Сообщение с ошибкой:      [0 0 0 0 0 1 0]
Исправленное сообщение:   [1 1 0 0 0 1 0]
Ошибка исправлена неудачно


# **6.2**

In [53]:
# Введем новые входное сообщение a и порождающий полином g = 1 + x^3 + x^4 + x^5 + x^6
a = np.array([1, 0, 0, 1, 0, 0, 0, 1, 1])
g = np.array([1, 0, 0, 1, 1, 1, 1])

In [54]:
generate_and_correct_error(a, g, 1) # исследование для однократной ошибки

Исходное сообщение:       [1 0 0 1 0 0 0 1 1]
Порождающий полином:      [1 0 0 1 1 1 1]
Отправленное сообщение:   [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибкой:      [1 0 0 0 1 1 0 0 0 1 1 0 1 0 1]
Исправленное сообщение:   [1 0 0 0 1 1 0 0 0 1 1 1 1 0 1]
Ошибка исправлена неудачно


In [55]:
generate_and_correct_error(a, g, 2) # исследование для двухкратной ошибки

Исходное сообщение:       [1 0 0 1 0 0 0 1 1]
Порождающий полином:      [1 0 0 1 1 1 1]
Отправленное сообщение:   [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибкой:      [1 0 0 0 1 1 0 0 1 1 0 0 0 0 1]
Исправленное сообщение:   [1 0 0 0 1 1 0 1 1 0 0 0 0 0 1]
Ошибка исправлена неудачно


In [56]:
generate_and_correct_error(a, g, 3) # исследование для трехкратной ошибки

Исходное сообщение:       [1 0 0 1 0 0 0 1 1]
Порождающий полином:      [1 0 0 1 1 1 1]
Отправленное сообщение:   [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибкой:      [1 0 0 0 1 1 0 0 1 1 1 1 0 1 1]
Исправленное сообщение:   [1 0 0 0 1 1 0 0 1 1 1 0 0 0 1]
Ошибка исправлена неудачно


In [57]:
generate_and_correct_error(a, g, 4) # исследование для четырехкратной ошибки

Исходное сообщение:       [1 0 0 1 0 0 0 1 1]
Порождающий полином:      [1 0 0 1 1 1 1]
Отправленное сообщение:   [1 0 0 0 1 1 0 0 0 1 1 0 0 0 1]
Сообщение с ошибкой:      [1 0 1 0 1 1 1 0 0 1 1 0 1 1 1]
Исправленное сообщение:   [1 0 1 0 1 1 1 0 1 1 1 0 1 1 1]
Ошибка исправлена неудачно
